In [5]:
import serial
import matplotlib.pyplot as plt
import serial.tools.list_ports as list_ports
from IPython.display import clear_output
import time
import math

import pandas as pd

# import plotext as plt
from ast import literal_eval as make_tuple

PID_MICROBIT = 516
VID_MICROBIT = 3368
TIMEOUT = 0.1

In [6]:
def find_comport(pid, vid, baud):
    ''' return a serial port '''
    ser_port = serial.Serial(timeout=TIMEOUT)
    ser_port.baudrate = baud
    ports = list(list_ports.comports())
    print('scanning ports')
    for p in ports:
        print('port: {}'.format(p))
        try:
            print('pid: {} vid: {}'.format(p.pid, p.vid))
        except AttributeError:
            continue
        if (p.pid == pid) and (p.vid == vid):
            print('found target device pid: {} vid: {} port: {}'.format(
                p.pid, p.vid, p.device))
            ser_port.port = str(p.device)
            return ser_port
    return None

In [7]:
def main():
    print('looking for microbit')
    ser_micro = find_comport(PID_MICROBIT, VID_MICROBIT, 115200)
    if not ser_micro:
        print('microbit not found')
        return
    print('opening and monitoring microbit port')
    ser_micro.open()
    while True:
        line = ser_micro.readline().decode('utf-8')
        if "C" in line:
            break
        if "B" in line:
            break
        line = line.split(" A")[0]
        try:
            acc = make_tuple(line)
            ax = int(acc[0])
            ay = int(acc[1])
            az = int(acc[2])
            total_acc = math.sqrt(ax**2 + ay**2 + az**2)
            a.append(total_acc)
            if len(a) >= 20:
                d20 = sum(a[len(a)-20:])/20
                ma20.append(d20)
            
            if len(a) >= 250:
                d250 = sum(a[len(a)-250:])/250
                ma250.append(d250)
                d2f250 = abs(d250-d20)
                diffma.append(d2f250)
                avgA.append(sum(diffma[len(diffma)-20:])/20)
                
            if len(a) >= 300:
                clear_output(wait=True)
                averageOf30diff = sum(diffma[len(diffma)-300:])/300
                if averageOf30diff > 20:
                    if averageOf30diff > 150:
                        print("falling!!!")
                    else:
                        print("walking")
                else:
                    print("rest")
                
#                 pitch = math.atan(ax / (math.sqrt(ay**2 + az**2)))
#                 pitch = pitch * 180 / math.pi
#                 if( pitch < 0 ):
#                     pitch += 360;
#                 print("pitch:",pitch)
                
                yaw = math.atan(ay / (math.sqrt(ax**2 + az**2)))
                yaw = yaw * 180 / math.pi
                if( yaw < 0 ):
                    yaw += 360;
                print("yaw:",yaw)
                if 60 <= yaw <= 90:
                    print("upright")
                elif 45 <= yaw < 60:
                    print("tilted")
                else:
                    print("lying down")
                
#                 roll = math.atan((math.sqrt(ax**2 + ay**2))/ az)
#                 roll = roll * 180 / math.pi
#                 if( roll < 0 ):
#                     roll += 360;
#                 print("roll:",roll)
            else:
                print("calibrating")
                
        except Exception as e: 
            print(e)
#             break

    ser_micro.close()

In [8]:
x = []
y = []
z = []
a = []
ma20 = [1000 for i in range(19)]
ma250 = [1000 for i in range(249)]
diffma = [0 for i in range(249)]
avgA = [0 for i in range(249)]
avgA300 = [0 for i in range(299)]
main()

rest
yaw: 352.8478210672643
lying down
